In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np

In [ ]:
# Charger les données et ne conserver que les lignes où la liste des mots-clés n'est pas nulle
df = pd.read_excel('data_MHL.xlsx', index_col='Unnamed: 0').reset_index(drop=True)
df = df.dropna(subset=['Mots-clés principaux : ', 'Mots-clés secondaires : '])

# Faire la liste des mots-clés apparaissant à chaque ligne
keywords, all_kw = [], []
for i, row in df.iterrows():
    kw = []
    for key in ['Mots-clés principaux : ', 'Mots-clés secondaires : ']:
        kw += row[key].split(', ')
    all_kw += kw
    keywords.append(kw)
    
# Ordonner les mots-clés présents au moins 3 fois dans le dataset dans un tableau numéroté
one_hot_labels = pd.Series(np.array(all_kw)).value_counts()
one_hot_labels = one_hot_labels[one_hot_labels >= 3].keys()

In [ ]:
# Encoder chaque entité dans une vecteur binaire (one-hot encoding)
one_hot = np.zeros((len(keywords), len(one_hot_labels)))
for i, list_kw in tqdm.tqdm(enumerate(keywords)):
    for kw in list_kw:
        one_hot[i][int(np.argmax(one_hot_labels == kw))] = 1

In [ ]:
# Algorithme de Latent Dirichlet allocation
lda = LatentDirichletAllocation(n_components=8, random_state=0)
res = lda.fit_transform(one_hot)

# Afficher la perplexité
print(lda.perplexity(one_hot))

# Evaluer les thèmes pour chaque mot-clé unique
kw_class = lda.transform(np.identity(len(one_hot_labels)))

# Afficher la répartition des images par thème
pd.Series(np.argmax(res, axis=1)).value_counts().sort_index()/(0.01*len(df))

In [ ]:
# Afficher le top 10 des tags les plus spécifiques à chaque thème
for j in range(8):
    print('Thème', j)
    for l in one_hot_labels[np.flip(np.argsort(kw_class[:, j]))[:10]]:
        print(l.capitalize())

In [ ]:
# Afficher (si disponible) l'url du top 10 des images les plus spécifiques à chaque thème
for j in np.flip(np.argsort(res[:,7])[-10:]):
    url = df['image_url'][df.index[j]]
    if type(url) == str:
        print(url)